# Torch sanity checks
> Verifying that the `TorchSolver` class is working as expected.

In [2]:
import sys
sys.path.append('/home/phil/aptr')
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np

from src.torch_solver import TorchSolver
from src.database import RnaDB

from src.solve_table import solve_all, score_predictions
from src.simulation import simulate_from_ids

In [8]:
# Load a database object

db = RnaDB()

/home/phil/miniconda3/envs/aPTR/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/phil/miniconda3/envs/aPTR/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [9]:
# Get 10 complete genomes

genomes = db.db[db.db["n_contigs"] == 1]["genome"].unique()
genomes_to_use = np.random.choice(genomes, 10, replace=False)

# Simulate reads from those genomes
reads, ptrs, coverages, otus = simulate_from_ids(
    db=db.db,
    ids=genomes_to_use,
    fasta_path="/home/phil/aptr/data/seqs",
    n_samples=10,
    scale=1e6,
    shuffle=False,  # Suppress shuffling to conserve memory
    fastq=False
)


NameError: name 'np' is not defined

In [6]:
# Here we initialize a TorchSolver with the genomes and coverages from before

solver = TorchSolver()
solver.set_vals(
    genomes=genomes,
    coverages=coverages
)

NameError: name 'genomes' is not defined